In [1]:
import nltk
import pandas as pd
import numpy as np
import numbers
from num2words import num2words
import collections
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import nltk.collocations
import nltk.corpus
import collections
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet
from nltk.tag.stanford import StanfordPOSTagger

In [15]:
import requests

def getWikiSenses(word):
    
    #create a connection(session)
    r_session = requests.Session()

    #url for the MediaWiki action API
    URL = "https://en.wikipedia.org/w/api.php"

    PARAMS = {
        "action": "query", #we are creating a query
        "titles": word, #for the title input param    
        "prop": "redirects", #asking for all the redirects (to the title car)
        "format": "json" #and we want the output in a json format
    }

    #we obtain the response to the get request with the given parmeters
    query_response = r_session.get(url=URL, params=PARAMS)
    json_data = query_response.json()

    #print(json_data)
    wikipedia_pages = json_data["query"]["pages"]

    #we iterate through items and print all the redirects (their title and id)
    try:
        L = []
        for k, v in wikipedia_pages.items():
            for redir in v["redirects"]:
                if redir["title"].isalpha():             
                    L.append((redir["title"], redir["pageid"]))
                
        return L
    except KeyError as err:
        if err.args[0]=='redirects':
            
            return []
        else:
            print(repr(err))

In [16]:
WordNet_senses = wordnet.synsets('tree')
print(WordNet_senses)

[Synset('tree.n.01'), Synset('tree.n.02'), Synset('tree.n.03'), Synset('corner.v.02'), Synset('tree.v.02'), Synset('tree.v.03'), Synset('tree.v.04')]


In [17]:
Wikipedia_senses = getWikiSenses('tree')
print(Wikipedia_senses)

[('Trees', 546743), ('Sapling', 982234), ('Arboreus', 9996504), ('Macrophanerophyte', 23637495), ('Treee', 41412516)]


In [19]:
map_ = {}
for w in Wikipedia_senses:
    map_[w] = None

In [27]:
for w in Wikipedia_senses:
    
    wiki = getWikiSenses(w[1])
    wn = wordnet.synsets(w[0])
    
    if len(wiki) == 1  and len(wn) == 1:
        map_[w] = wn[0]

In [29]:
for w in Wikipedia_senses:
    
    if map_[w] is None:
        wiki = getWikiSenses(w[1])
        
        for d in wiki:
            if map_[d] is not None and map_[w] in wordnet.synsets(w[0]):
                map_[w] = map_[d]
                break

In [30]:
print(map_)

{('Trees', 546743): None, ('Sapling', 982234): None, ('Arboreus', 9996504): None, ('Macrophanerophyte', 23637495): None, ('Treee', 41412516): None}
